In [1]:
#Import all necessary packages
import pandas as pd
import numpy as np
import datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
import pvlib
import pvanalytics
import pvpltools
import plantpredict
import math
import time
import scipy
import seaborn as sns

#Import all necessary modules from packages
from datetime import datetime
from pvpltools import iec61853
from pvpltools.module_efficiency import adr, heydenreich, motherpv, pvgis, mpm5, mpm6, bilinear
from pvpltools.module_efficiency import fit_efficiency_model, fit_bilinear
from pandas.plotting import register_matplotlib_converters
from plantpredict.enumerations import PredictionStatusEnum, TranspositionModelEnum, SpectralShiftModelEnum,DiffuseDirectDecompositionModelEnum, ModuleTemperatureModelEnum, IncidenceAngleModelTypeEnum, AirMassModelTypeEnum, DirectBeamShadingModelEnum, SoilingModelTypeEnum, DegradationModelEnum, TrackingTypeEnum, BacktrackingTypeEnum, DiffuseShadingModelEnum
register_matplotlib_converters()
from scipy.optimize import minimize

#Set style / plotting preferences
mpl.rcParams['font.size']=12
mpl.rcParams['lines.linewidth']=1
mpl.rcParams['xtick.labelsize']=10
mpl.rcParams['axes.titlesize']=22
sns.set(rc={'figure.figsize':(12,7)})
sns.set_style('whitegrid')

In [2]:
api = plantpredict.Api(
    username="lelia.deville1@louisiana.edu.plantpredictapi",
    password="r9HBf70PC6dnr/kp3dwc9eBIzYQWMMP+7ekjM+EIXU2jl5pJf+AQ9/ZRVz6AAJUrad6vZ+KWZaoeTwsZHg65Sg==",
    client_id="0oadalz1takAapQQx357",
    client_secret="mEoW-QTKXPmzLPavJfMCymXD9gsqjQ9-bHhCoFJg")

In [3]:
project_id = 76406   # CHANGE TO YOUR PROJECT ID
prediction_id = 425992 # CHANGE TO YOUR PREDICTION ID
prediction = api.prediction(id=prediction_id, project_id=project_id)
project = api.project(id=project_id)

In [4]:
prediction.get()
project.get()

{'latitude': 30.22,
 'longitude': -92.02,
 'country': 'United States',
 'country_code': 'US',
 'state_province': 'Louisiana',
 'state_province_code': 'LA',
 'locality': 'Lafayette',
 'region': 'North America',
 'elevation': 10.11289024353027,
 'standard_offset_from_utc': -6.0,
 'predictions': [{'powerplant': None,
   'status': 1,
   'start': '2005-01-01T00:00:00',
   'end': '2005-12-31T23:00:00',
   'last_published_date_to_queue': '2021-11-21T15:51:18.313',
   'start_index': 0,
   'end_index': 0,
   'year_repeater': 1,
   'transposition_model': 1,
   'diffuse_direct_decomp_model': 3,
   'diffuse_direct_decomp_model_executed': False,
   'use_meteo_dni': False,
   'use_meteo_poai': False,
   'use_leap_years': False,
   'enable_le_t_i_d': False,
   'air_mass_model': 0,
   'soiling_model': 0,
   'mod_temp_model': 0,
   'inc_angle_model': 5,
   'direct_beam_shading_model': 0,
   'degradation_model': 3,
   'diffuse_shading_model': 1,
   'linear_degradation_rate': 0.5,
   'first_year_degradat

In [5]:
powerplant = api.powerplant(project_id=project.id, prediction_id=prediction.id)

In [6]:
powerplant.get()

{'id': 577642,
 'result_list': None,
 'blocks': [{'name': 1,
   'nodal_export_options': None,
   'result_list': None,
   'arrays': [{'name': 1,
     'result_list': None,
     'inverters': [{'name': 'A',
       'result_list': None,
       'dc_fields': [{'name': 1,
         'module_id': 298,
         'module': {'status': 6,
          'model': 'FS-4115-3 Feb 2016',
          'manufacturer': 'First Solar',
          'length': 1200.0,
          'width': 600.0,
          'weight': 12.0,
          'default_orientation': 0,
          'number_of_cells_in_series': 108,
          'number_of_cells_in_parallel': 2,
          'stc_short_circuit_current': 1.83,
          'stc_open_circuit_voltage': 87.6,
          'stc_mpp_current': 1.66,
          'stc_mpp_voltage': 69.3,
          'stc_max_power': 115.0,
          'stc_power_temp_coef': -0.28,
          'stc_short_circuit_current_temp_coef': 0.038,
          'stc_open_circuit_voltage_temp_coef': -0.28,
          'stc_efficiency': 15.97,
          '

In [7]:
prediction.run()

{'is_successful': True}

In [13]:
def objective(x,sign=-1.0):
    tilt = x[0]
    mod_in_series = x[1]
    dc_to_ac = x[2]
    
    
    
    powerplant.blocks[0]['arrays'][0]['inverters'][0]['dc_fields'][0]['module_tilt'] = tilt
    powerplant.blocks[0]['arrays'][0]['inverters'][0]['dc_fields'][0]['modules_wired_in_series']=int(mod_in_series)
    powerplant.desired_dc_ac_ratio = dc_to_ac
                                                                     
    powerplant.update()
    prediction.run()
    
    a = prediction.get_results_summary()
    a = a['years'][0]['summary_factors'][0]['plant_net_energy']
    return sign*a
    

In [14]:
bt = (20,50)
bm = (8,10)
br = (0.85,1.5)
bnds = (bt,bm,br)

In [15]:
for i in [8,9,10]:
    x0 = [25,i,1.2]
    sol = minimize(fun=objective,x0=x0, method='L-BFGS-B', bounds=bnds, options={'maxiter':6})
    print(sol)

      fun: -9.421353048427891
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([-1.97175593e-05,  0.00000000e+00, -0.00000000e+00])
  message: 'STOP: TOTAL NO. of ITERATIONS REACHED LIMIT'
     nfev: 40
      nit: 6
     njev: 10
   status: 1
  success: False
        x: array([23.08301083,  8.        ,  1.5       ])
      fun: -10.359422763318726
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 3.64153122e-05,  0.00000000e+00, -0.00000000e+00])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 48
      nit: 4
     njev: 12
   status: 0
  success: True
        x: array([23.61175148,  9.        ,  1.5       ])
      fun: -11.070453259913624
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 3.55271338e-07,  0.00000000e+00, -0.00000000e+00])
  message: 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 32
      nit: 4
     njev: 8
   status: 0
  success: True
        x: array([23.013